In [ ]:
!pip install kaggle
!pip install tensorflow


In [ ]:
import os
import zipfile
import pandas as pd


In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json



In [ ]:
! kaggle datasets download -d pankrzysiu/cifar10-python

In [ ]:

with zipfile.ZipFile('/content/cifar10-python.zip', 'r') as zip_ref:
    zip_ref.extractall('/content/cifar10-python')


Load and PreProcess the Dataset

In [ ]:
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split


In [ ]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

In [ ]:
# Normalize pixel values to the range [0, 1]
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

# Convert labels to one-hot encoded vectors
y_train = to_categorical(y_train, num_classes=10)
y_test = to_categorical(y_test, num_classes=10)


Split data to train and testing


In [ ]:

x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.1, random_state=42)

Build a CNN Model

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout


Model Creation

In [ ]:

model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)),
    MaxPooling2D((2, 2)),

    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),

    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),

    Flatten(),

    Dense(128, activation='relu'),

    Dropout(0.3),

    Dense(10, activation='softmax')
])

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


Train the model

In [ ]:
history = model.fit(x_train, y_train, epochs=10, batch_size=64, validation_data=(x_val, y_val))


In [ ]:
test_loss, test_acc = model.evaluate(x_test, y_test)
print("Test accuracy:", test_acc)


In [ ]:
import tensorflow as tf

# Assuming 'model' is your trained object detection model
model.save('/content/model.h5')


Image as an Input

In [ ]:
import numpy as np
from PIL import Image
import requests

In [ ]:
class_names = [
    'airplane', 'automobile', 'bird', 'cat', 'deer',
    'dog', 'frog', 'horse', 'ship', 'truck'
]


In [ ]:
def predict_image(image_path, model):
    from tensorflow.keras.preprocessing import image
    import numpy as np

    # Load and preprocess the image
    img = image.load_img(image_path, target_size=(32, 32))
    img = image.img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = img / 255.0

    # Make predictions
    predictions = model.predict(img)
    class_index = np.argmax(predictions)
    class_name = class_names[class_index]  # Get the class name

    # Print the predicted class name
    print("Predicted class:", class_name)

    return class_name

In [ ]:
image_path='/content/cat.jpg'

In [ ]:
predicted_class = predict_image(image_path, model)
print("Predicted class:", predicted_class)